In [2]:
import unittest
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist


class TestFashionMNISTModel(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        """Load data and set up necessary components once for all tests"""
        (cls.X_train, cls.y_train), (cls.X_test, cls.y_test) = fashion_mnist.load_data()
        cls.img_width, cls.img_height = 224, 224
        cls.num_classes = 10

    def test_one_hot_encoding(self):
        """Test if labels are correctly one-hot encoded"""
        y_train_encoded = to_categorical(self.y_train, self.num_classes)
        y_test_encoded = to_categorical(self.y_test, self.num_classes)

        # Check if the output is in the correct shape
        self.assertEqual(y_train_encoded.shape[1], self.num_classes)
        self.assertEqual(y_test_encoded.shape[1], self.num_classes)

    def test_data_preprocessing(self):
        """Test if the data preprocessing correctly resizes and converts images to RGB"""
        def preprocess(image, label):
            image = tf.image.grayscale_to_rgb(image)
            image = tf.image.resize(image, (self.img_width, self.img_height))
            return image, label

        X_train_reshaped = self.X_train.reshape(-1, 28, 28, 1)
        X_test_reshaped = self.X_test.reshape(-1, 28, 28, 1)

        X_train_tensor = tf.convert_to_tensor(X_train_reshaped, dtype=tf.float32)
        X_test_tensor = tf.convert_to_tensor(X_test_reshaped, dtype=tf.float32)

        # Preprocess a sample
        preprocessed_image, _ = preprocess(X_train_tensor[0], self.y_train[0])

        # Check if the image has been resized to 224x224
        self.assertEqual(preprocessed_image.shape, (self.img_width, self.img_height, 3))

    def test_model_architecture(self):
        """Test if the model architecture is built correctly"""
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(self.img_width, self.img_height, 3))
        base_model.trainable = False

        # Add custom layers
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.5)(x)
        outputs = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=outputs)

        # Check if the output shape is correct
        self.assertEqual(model.output_shape, (None, self.num_classes))


if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)  # Use argv=[''] to avoid IPython and Jupyter argv issues


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


.

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


..
----------------------------------------------------------------------
Ran 3 tests in 4.721s

OK
